In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 1. 공통설정

In [2]:
import os
import json
import glob
import pandas as pd
from datasets import Dataset
from dotenv import load_dotenv
import gc  # 메모리 관리
load_dotenv()

from openai import OpenAI
from ragas.llms import llm_factory
from langchain_openai import OpenAIEmbeddings 
from langchain_upstage import ChatUpstage, UpstageEmbeddings

openai_client = OpenAI()
ragas_llm = llm_factory(
    model="gpt-4o-mini",
    provider="openai",
    client=openai_client,
)

# ragas_embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-small",
#     openai_api_key=os.getenv("OPENAI_API_KEY")
# )

# 평가용 임베딩 Upstage로 
ragas_embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

print("✅ ragas_llm ready:", type(ragas_llm))
print("✅ ragas_embeddings ready:", type(ragas_embeddings))

# embedding 테스트
try:
    test_vec = ragas_embeddings.embed_query("테스트")
    print(f"✅ Embedding test successful! Dimension: {len(test_vec)}")
except Exception as e:
    print(f"❌ Embedding test failed: {e}")


# 최신 캐시 jsonl 찾기
def find_latest_cache(pattern="./data/RAGAS/rag_run_cache_*.jsonl") -> str:
    candidates = glob.glob(pattern)
    if not candidates:
        raise FileNotFoundError(f"캐시 파일이 없습니다: {pattern}")
    return max(candidates, key=os.path.getmtime)

# CACHE_JSONL = find_latest_cache()
# print("✅ latest cache:", CACHE_JSONL)

# jsonl 로드
# def load_jsonl(path: str):
#     rows = []
#     with open(path, "r", encoding="utf-8") as f:
#         for line in f:
#             if line.strip():
#                 rows.append(json.loads(line))
#    return rows

# raw_rows = load_jsonl(CACHE_JSONL)

# 컬럼명 표준화 + 데이터 검증 강화
# def normalize_row_schema(rows):
#     out = []
#     for i, r in enumerate(rows):
#         rr = dict(r)

#         # question/user_input 통일
#         if "question" not in rr and "user_input" in rr:
#             rr["question"] = rr["user_input"]
        
#         # ✅ question이 비어있거나 None이면 스킵
#         if not rr.get("question"):
#             print(f"⚠️ Warning: Row {i} has empty question, skipping")
#             continue

#         # ✅ answer가 비어있거나 None이면 기본값 설정
#         if not rr.get("answer"):
#             print(f"⚠️ Warning: Row {i} has empty answer, using default")
#             rr["answer"] = "답변을 생성할 수 없습니다."

#         # contexts 통일
#         if "contexts" not in rr and "retrieved_contexts" in rr:
#             rr["contexts"] = rr["retrieved_contexts"]
        
#         # ✅ contexts가 없으면 빈 리스트로 초기화
#         if "contexts" not in rr:
#             rr["contexts"] = []
        
#         # ✅ contexts가 리스트가 아니면 변환
#         if not isinstance(rr.get("contexts"), list):
#             rr["contexts"] = [str(rr["contexts"])] if rr["contexts"] else []

#         # reference 통일
#         if "reference" not in rr:
#             gts = rr.get("ground_truths", None)
#             if isinstance(gts, list) and len(gts) > 0:
#                 rr["reference"] = gts[0]
#             elif "ground_truth" in rr:
#                 rr["reference"] = rr["ground_truth"]
#             else:
#                 rr["reference"] = ""  # ✅ 기본값 설정

#         out.append(rr)
#     return out

# rows = normalize_row_schema(raw_rows)
# print(f"✅ Normalized rows: {len(rows)}")

# ✅ 데이터 검증 함수 추가
def validate_data(rows, required_fields):
    """데이터 검증 및 문제 리포트"""
    issues = []
    for i, r in enumerate(rows):
        for field in required_fields:
            if field not in r:
                issues.append(f"Row {i}: missing field '{field}'")
            elif r[field] is None:
                issues.append(f"Row {i}: field '{field}' is None")
            elif field in ["question", "answer"] and not str(r[field]).strip():
                issues.append(f"Row {i}: field '{field}' is empty")
    
    if issues:
        print("⚠️ Data validation issues:")
        for issue in issues[:10]:  # 처음 10개만 출력
            print(f"  - {issue}")
        if len(issues) > 10:
            print(f"  ... and {len(issues) - 10} more issues")
    else:
        print("✅ Data validation passed")
    
    return len(issues) == 0

# 컨텍스트 클립 개선
def clip_rows(rows, *, max_ctx=3, max_chars=600, clip_reference_chars=None, clip_answer_chars=None):
    clipped = []
    for r in rows:
        rr = dict(r)

        ctx = rr.get("contexts", []) or []
        # ✅ None 값 필터링 추가
        ctx = [c for c in ctx if c is not None]
        rr["contexts"] = [str(c)[:max_chars] for c in ctx[:max_ctx]]

        if clip_reference_chars is not None and rr.get("reference"):
            rr["reference"] = str(rr["reference"])[:clip_reference_chars]

        if clip_answer_chars is not None and rr.get("answer"):
            rr["answer"] = str(rr["answer"])[:clip_answer_chars]

        clipped.append(rr)
    return clipped

# ✅ 저장 폴더
OUT_DIR = "./data/RAGAS"
os.makedirs(OUT_DIR, exist_ok=True)

def save_result(result, metric_name: str) -> str:
    """
    result: ragas evaluate result
    metric_name: 파일명에 박을 이름
    """
    df = result.to_pandas()
    
    # ✅ 결과 요약 출력
    print(f"\n📊 {metric_name} Results:")
    print(f"  Mean: {df[metric_name].mean():.4f}")
    print(f"  Std: {df[metric_name].std():.4f}")
    print(f"  Min: {df[metric_name].min():.4f}")
    print(f"  Max: {df[metric_name].max():.4f}")
    print(f"  NaN count: {df[metric_name].isna().sum()}")
    
    out_path = os.path.join(OUT_DIR, f"ragas_result_{metric_name}.csv")
    df.to_csv(out_path, index=False, encoding="utf-8")
    print(f"✅ saved: {out_path}\n")
    return out_path

# ✅ 메모리 정리 함수
def cleanup_memory():
    """메모리 정리"""
    gc.collect()
    print("🧹 Memory cleaned")

✅ ragas_llm ready: <class 'ragas.llms.base.InstructorLLM'>
✅ ragas_embeddings ready: <class 'langchain_upstage.embeddings.UpstageEmbeddings'>
✅ Embedding test successful! Dimension: 4096


# 2. 함수실행

In [3]:
import os
import re
import sys
import json
import numpy as np
import pandas as pd
from typing import List, Dict, Any

from docx import Document as DocxDocument
from datasets import Dataset

from ragas import evaluate
from ragas.metrics._context_precision import ContextPrecision
from ragas.metrics._context_recall import ContextRecall

import plotly.express as px
import plotly.graph_objects as go

# -----------------------------
# 0) RAG 모듈 import (프로젝트 구조 그대로)
# -----------------------------
MODULES_PATH = os.path.join(os.getcwd(), "chatbot_app", "modules")
sys.path.append(MODULES_PATH)
sys.path.append(os.getcwd())
from chatbot_app.modules.rag_module import create_pipeline, RAGConfig

# -----------------------------
# 1) 입력: 질문 docx 경로
# -----------------------------
DOC_PATH = "./data/RAGAS/RAGAS_질문_10개_정리본.docx"

def parse_ragas_docx(doc_path: str) -> List[Dict[str, Any]]:
    doc = DocxDocument(doc_path)
    paras = [p.text.strip() for p in doc.paragraphs if p.text and p.text.strip()]
    text = "\n".join(paras)

    blocks = re.split(r"\n(?=\d+\.\s)", text)

    items = []
    for b in blocks:
        b = b.strip()
        if not re.match(r"^\d+\.\s", b):
            continue

        parts = b.split("✔️ 모범답안", 1)
        if len(parts) != 2:
            continue

        q_part = re.sub(r"^\d+\.\s*", "", parts[0].strip())
        a_part = parts[1].strip()

        items.append({"question": q_part.strip(), "ground_truth": a_part.strip()})

    if len(items) == 0:
        raise ValueError("docx에서 문항을 1개도 파싱하지 못했습니다. 문서 포맷을 확인하세요.")
    return items

items = parse_ragas_docx(DOC_PATH)
print(f"✅ 파싱된 문항 수: {len(items)} (기대: 10)")

# -----------------------------
# 2) run_tag 생성 (파일명 식별용)
# -----------------------------
def make_run_tag(cfg: RAGConfig) -> str:
    rerank_flag = "T" if getattr(cfg, "enable_rerank", False) else "F"
    dw = getattr(cfg, "hybrid_dense_weight", None)
    sw = getattr(cfg, "hybrid_sparse_weight", None)
    k1 = getattr(cfg, "bm25_k1", None)
    dc = getattr(cfg, "bm25_max_doc_chars", None)
    return (
        f"s{cfg.search_multiplier}"
        f"_law{cfg.k_law}_rule{cfg.k_rule}_case{cfg.k_case}"
        f"_dw{dw}_sw{sw}_k1{k1}_dc{dc}"
        f"_rerank{rerank_flag}"
    )

# -----------------------------
# 3) 저장 루트 (요구사항: ./data/RAGAS/ 밑)
# -----------------------------
RUNS_ROOT = os.path.join(OUT_DIR, "runs")
os.makedirs(RUNS_ROOT, exist_ok=True)

# -----------------------------
# 4) R_i 가중치 (요구사항대로)
# -----------------------------
W_RECALL = 0.6
W_PREC   = 0.4

# -----------------------------
# 5) 캐시(jsonl) 생성 + RAGAS 평가 1회(run_once)
# -----------------------------
def run_once(cfg: RAGConfig, items: List[Dict[str, Any]], run_idx: int) -> Dict[str, Any]:
    run_tag = make_run_tag(cfg)
    run_dir = os.path.join(RUNS_ROOT, run_tag, f"run_{run_idx:02d}")
    os.makedirs(run_dir, exist_ok=True)

    cache_jsonl = os.path.join(run_dir, "rag_run_cache.jsonl")

    # (1) pipeline 생성
    pipeline = create_pipeline(config=cfg)

    # (2) 캐시 저장 (jsonl)
    with open(cache_jsonl, "w", encoding="utf-8") as f:
        for i, ex in enumerate(items, start=1):
            q = ex["question"]
            gt = ex["ground_truth"]

            
            print(f"[run_{run_idx:02d}] Q{i} chars=", len(q))
            if len(q) > 2000:
                print("[WARN] very long question head:", q[:200])

            MAX_QUERY_CHARS = 2000  # 안전값(너무 보수적이면 1500~2000까지 올려도 됨)

            q_for_retrieval = q if len(q) <= MAX_QUERY_CHARS else q[:MAX_QUERY_CHARS]
            
            try:
                trace = pipeline.answer_with_trace(q_for_retrieval, skip_normalization=False)
            except Exception as e:
                print(f"[ERROR] run_{run_idx:02d} Q{i} retrieval failed:", repr(e))
                trace = {"answer": "", "docs": [], "normalized_query": ""}

            answer = trace.get("answer", "") or ""
            docs = trace.get("docs", []) or []

            contexts = []
            for d in docs:
                try:
                    contexts.append(d.page_content)
                except Exception:
                    pass

            row = {
                "id": i,
                "question": q,
                "answer": answer,
                "contexts": contexts,
                "reference": gt,
                "ground_truths": [gt],
                "normalized_query": trace.get("normalized_query", ""),
            }
            f.write(json.dumps(row, ensure_ascii=False) + "\n")

    # (3) ragas용 rows 생성 (ragas공통.txt 스키마를 그대로 사용)
    raw_rows = []
    with open(cache_jsonl, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                raw_rows.append(json.loads(line))

    # ragas공통.txt에 있는 normalize_row_schema()를 그대로 쓰는 대신,
    # 동일한 컬럼키(question/answer/contexts/reference)를 이미 넣었으므로 최소 정리만 수행
    rows_local = []
    for r in raw_rows:
        rr = dict(r)
        if not rr.get("question"):
            continue
        if not rr.get("answer"):
            rr["answer"] = "답변을 생성할 수 없습니다."
        if "contexts" not in rr or rr["contexts"] is None:
            rr["contexts"] = []
        if not isinstance(rr["contexts"], list):
            rr["contexts"] = [str(rr["contexts"])]
        if "reference" not in rr:
            rr["reference"] = ""
        rows_local.append(rr)

    # 데이터 검증(함수는 ragas공통.txt에서 제공)
    validate_data(rows_local, required_fields=["question", "contexts"])

    # ✅ 중요: context_precision / context_recall 입력 클립을 분리
    rows_cp = clip_rows(rows_local, max_ctx=3, max_chars=600)
    rows_cr = clip_rows(
        rows_local,
        max_ctx=2,
        max_chars=200,
        clip_reference_chars=300,
        clip_answer_chars=300,
    )

    ds_p = Dataset.from_list([
        {
            "question": r["question"],
            "answer": r["answer"],
            "contexts": r.get("contexts", []),
            "reference": r.get("reference", ""),
        }
        for r in rows_cp
    ])

    ds_r = Dataset.from_list([
        {
            "question": r["question"],
            "answer": r["answer"],
            "contexts": r.get("contexts", []),
            "reference": r.get("reference", ""),
        }
        for r in rows_cr
    ])

# (4) RAGAS 평가: precision / recall
    res_p = evaluate(
        dataset=ds_p,
        metrics=[ContextPrecision()],
        llm=ragas_llm,
        embeddings=ragas_embeddings,
        show_progress=True,
        raise_exceptions=True,
        batch_size=1,
    )
    df_p = res_p.to_pandas()
    p_csv = os.path.join(run_dir, "ragas_result_context_precision.csv")
    df_p.to_csv(p_csv, index=False, encoding="utf-8")

    res_r = evaluate(
        dataset=ds_r,
        metrics=[ContextRecall()],
        llm=ragas_llm,
        embeddings=ragas_embeddings,
        show_progress=True,
        raise_exceptions=True,
        batch_size=1,
    )
    df_r = res_r.to_pandas()
    r_csv = os.path.join(run_dir, "ragas_result_context_recall.csv")
    df_r.to_csv(r_csv, index=False, encoding="utf-8")

    # (5) per-question merge → R_i (행: 1~10)
    # id 컬럼이 없다면 보정
    if "id" not in df_p.columns:
        df_p = df_p.copy()
        df_p["id"] = range(1, len(df_p) + 1)
    if "id" not in df_r.columns:
        df_r = df_r.copy()
        df_r["id"] = range(1, len(df_r) + 1)

    dfm = pd.merge(
        df_r[["id", "context_recall"]],
        df_p[["id", "context_precision"]],
        on="id",
        how="inner",
    )
    dfm["R_i"] = (W_RECALL * dfm["context_recall"].astype(float) + W_PREC * dfm["context_precision"].astype(float))

    # 저장용 2자리
    dfm_save = dfm.copy()
    for c in ["context_recall", "context_precision", "R_i"]:
        dfm_save[c] = dfm_save[c].astype(float).round(2)

    perq_csv = os.path.join(run_dir, "per_question.csv")
    dfm_save.to_csv(perq_csv, index=False, encoding="utf-8")

    cleanup_memory()

    return {
        "run_tag": run_tag,
        "run_idx": run_idx,
        "run_dir": run_dir,
        "cache_jsonl": cache_jsonl,
        "precision_csv": p_csv,
        "recall_csv": r_csv,
        "per_question_csv": perq_csv,
        "df_perq": dfm_save,
        "R_mean": float(dfm["R_i"].mean()),
        "R_std": float(dfm["R_i"].std(ddof=1)) if len(dfm) > 1 else 0.0,
    }
    print(f"[run{run_idx}] Q{i} q_len={len(q)} retrieval_len={len(q_for_retrieval)}")


# -----------------------------
# 6) 반복 실행(run_repeat): cfg 튜닝용
# -----------------------------
def run_repeat(cfg: RAGConfig, items: List[Dict[str, Any]], n: int = 1) -> List[Dict[str, Any]]:
    results = []
    for run_idx in range(1, n + 1):
        print(f"\n==============================")
        print(f"RUN {run_idx:02d}/{n} | {make_run_tag(cfg)}")
        print(f"==============================")
        results.append(run_once(cfg, items, run_idx))
    return results

# -----------------------------
# 7) wide CSV 생성 (행: 1~10, 열: r1-precision/r1-recall/r1-Ri/...)
# -----------------------------
def build_wide_csv(run_tag: str, results: List[Dict[str, Any]]) -> str:
    """질문별 wide 테이블 + 마지막 mean 행 + Ri-mean(3회 평균) 컬럼 생성"""
    base_dir = os.path.join(RUNS_ROOT, run_tag)
    nq = len(items)
    ids = pd.DataFrame({"id": range(1, nq + 1)})
    wide = ids.copy()

    # run별 병합
    for res in results:
        ridx = res["run_idx"]
        dfm = res["df_perq"][["id", "context_precision", "context_recall", "R_i"]].copy()
        dfm = dfm.rename(columns={
            "context_precision": f"r{ridx}-precision",
            "context_recall": f"r{ridx}-recall",
            "R_i": f"r{ridx}-Ri",
        })
        wide = pd.merge(wide, dfm, on="id", how="left")

    wide = wide.set_index("id").sort_index()

    # 질문별 Ri 평균(3회)
    ri_cols = [c for c in wide.columns if c.endswith("-Ri")]
    wide["Ri-mean"] = wide[ri_cols].mean(axis=1, skipna=True)

    # 마지막 mean 행(열별 평균)
    mean_row = wide.mean(axis=0, skipna=True).to_frame().T
    mean_row.index = ["mean"]
    wide2 = pd.concat([wide, mean_row], axis=0)

    out_csv = os.path.join(base_dir, "runs_wide_3repeats.csv")
    wide2.to_csv(out_csv, encoding="utf-8")
    print("✅ saved:", out_csv)

    # 보기 좋게 소수 2자리로 출력(저장은 float 그대로)
    display(wide2.round(2))
    return out_csv


# -----------------------------
# 8) 3회 평균 기반 이미지(평균만) 저장
# -----------------------------
def save_png(fig, path: str):
    fig.write_image(path, scale=2)
    print("🖼️ saved:", path)

def make_mean_images(run_tag: str, results: List[Dict[str, Any]]):
    """3회 결과를 질문별 평균으로 합쳐서 bar/table PNG 1세트만 저장"""
    base_dir = os.path.join(RUNS_ROOT, run_tag)
    out_dir = os.path.join(base_dir, "mean")
    os.makedirs(out_dir, exist_ok=True)

    # results -> (id, recall/precision/R_i) 평균
    dfs = [r["df_perq"][["id", "context_recall", "context_precision", "R_i"]].copy() for r in results]
    merged = dfs[0].rename(columns={
        "context_recall": "r1_recall",
        "context_precision": "r1_precision",
        "R_i": "r1_Ri",
    })

    # 안전하게 run_idx 기준으로 붙이기
    nq = len(items)
    by_id = pd.DataFrame({"id": range(1, nq + 1)})
    for r in results:
        ridx = r["run_idx"]
        df = r["df_perq"][["id", "context_recall", "context_precision", "R_i"]].copy()
        df = df.rename(columns={
            "context_recall": f"r{ridx}_recall",
            "context_precision": f"r{ridx}_precision",
            "R_i": f"r{ridx}_Ri",
        })
        by_id = by_id.merge(df, on="id", how="left")

    recall_cols = [c for c in by_id.columns if c.endswith("_recall")]
    prec_cols = [c for c in by_id.columns if c.endswith("_precision")]
    ri_cols = [c for c in by_id.columns if c.endswith("_Ri")]

    mean_df = pd.DataFrame({
        "id": by_id["id"],
        "context_recall": by_id[recall_cols].mean(axis=1, skipna=True),
        "context_precision": by_id[prec_cols].mean(axis=1, skipna=True),
        "R_i": by_id[ri_cols].mean(axis=1, skipna=True),
    }).sort_values("id")

    mean_df["label"] = mean_df["id"].astype(str)

    # bar (고정순서 Q1->Q10)
    fig_bar = px.bar(
        mean_df,
        x="R_i",
        y="label",
        orientation="h",
        title=f"R_i per Question (mean of 3 runs, fixed order) | {run_tag}",
        text="R_i",
    )
    fig_bar.update_traces(texttemplate="%{text:.2f}", textposition="outside")
    fig_bar.update_layout(
        xaxis=dict(range=[0, 1.05]),
        yaxis=dict(autorange="reversed"),
        height=520
    )

    # table (소수 2자리 문자열로 표시)
    tdf = mean_df[["id", "context_recall", "context_precision", "R_i"]].copy()
    for c in ["context_recall", "context_precision", "R_i"]:
        tdf[c] = tdf[c].map(lambda x: f"{float(x):.2f}")

    fig_table = go.Figure(
        data=[go.Table(
            header=dict(values=list(tdf.columns)),
            cells=dict(values=[tdf[c].tolist() for c in tdf.columns]),
        )]
    )
    fig_table.update_layout(title="Per-question metrics (mean of 3 runs, 2 decimals)", height=420)

    bar_path = os.path.join(out_dir, "mean_runs_bar.png")
    tbl_path = os.path.join(out_dir, "mean_runs_table.png")
    save_png(fig_bar, bar_path)
    save_png(fig_table, tbl_path)
    return bar_path, tbl_path


✅ 파싱된 문항 수: 10 (기대: 10)


# 3. 튜닝

In [4]:
%%time
cfg = RAGConfig(
    k_law=7,             # 5
    k_rule=7,            # 5
    k_case=3,            # 3
    search_multiplier=4, # 2
    enable_rerank=True,
    hybrid_dense_weight=0.5,  # 0.6
    hybrid_sparse_weight=0.5, # 0.4
    bm25_k1=1.5,              # 1.5
    bm25_max_doc_chars=2200,  # 3000
)

results = run_repeat(cfg, items, n=3)

# wide CSV + 출력(+ mean 행, Ri-mean 컬럼)
wide_csv = build_wide_csv(results[0]["run_tag"], results)

# 3회 평균 기반 이미지 1세트만 저장
bar_path, tbl_path = make_mean_images(results[0]["run_tag"], results)
print("✅ MEAN images:", bar_path, tbl_path)

2026-02-06 12:54:06,046 - chatbot_app.modules.rag_module - INFO - 🔗 Pinecone 3중 인덱스 연결 중...



RUN 01/3 | s4_law7_rule7_case3_dw0.5_sw0.5_k11.5_dc2200_rerankT


2026-02-06 12:54:07,606 - chatbot_app.modules.rag_module - INFO - ✅ [Law / Rule / Case] 3개 인덱스 로드 완료!
2026-02-06 12:54:07,651 - chatbot_app.modules.rag_module - INFO - ℹ️ SimpleTokenizer 사용 (BM25)


[run_01] Q1 chars= 32


2026-02-06 12:54:08,249 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"
2026-02-06 12:54:08,256 - chatbot_app.modules.rag_module - INFO - 🔄 표준화된 질문: 주민등록·확정일자(확정일자) 했는데, 확정일자부(확정일자부) 내용까지 중요한가요?
2026-02-06 12:54:08,256 - chatbot_app.modules.rag_module - INFO - 🔍 [Hybrid Retrieval] query='주민등록·확정일자(확정일자) 했는데, 확정일자부(확정일자부) 내용까지 중요한가요?'
2026-02-06 12:54:10,661 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-06 12:54:13,276 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-06 12:54:15,580 - httpx - INFO - HTTP Request: POST https://api.upstage.ai/v1/solar/embeddings "HTTP/1.1 200 OK"
2026-02-06 12:54:17,834 - httpx - INFO - HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
2026-02-06 12:54:17,850 - chatbot_app.modules.rag_module - INFO - 📌 Rerank selected=2 (threshold=0.2)
2026-02-06 12:54:17,850 - chatbot

...

,r1-precision,r1-recall,r1-Ri,r2-precision,r2-recall,r2-Ri,r3-precision,r3-recall,r3-Ri,Ri-mean
1,1.0,0.43,0.66,1.0,0.00,0.40,1.0,0.57,0.74,0.60
2,1.0,0.86,0.91,1.0,0.14,0.49,1.0,0.14,0.49,0.63
3,1.0,0.88,0.92,1.0,0.86,0.91,1.0,0.88,0.92,0.92
4,1.0,0.78,0.87,1.0,0.82,0.89,1.0,0.90,0.94,0.90
5,1.0,0.80,0.88,1.0,0.25,0.55,1.0,0.80,0.88,0.77
6,1.0,0.64,0.78,1.0,0.64,0.78,1.0,1.00,1.00,0.85
7,1.0,0.22,0.53,1.0,0.22,0.53,1.0,0.40,0.64,0.57
8,1.0,0.40,0.64,1.0,0.77,0.86,1.0,0.40,0.64,0.71
9,1.0,0.50,0.70,1.0,0.50,0.70,1.0,0.50,0.70,0.70
10,1.0,0.75,0.85,1.0,0.60,0.76,1.0,0.75,0.85,0.82


2026-02-06 13:07:47,049 - choreographer.browsers.chromium - INFO - Chromium init'ed with kwargs {}
2026-02-06 13:07:47,055 - choreographer.browsers.chromium - INFO - Found chromium path: C:\Program Files\Google\Chrome\Application\chrome.exe
2026-02-06 13:07:47,056 - choreographer.utils._tmpfile - INFO - Temp directory created: C:\Users\Admin\AppData\Local\Temp\tmp1ojxzlvl.
2026-02-06 13:07:47,058 - choreographer.browser_async - INFO - Opening browser.
2026-02-06 13:07:47,059 - choreographer.utils._tmpfile - INFO - Temp directory created: C:\Users\Admin\AppData\Local\Temp\tmp16962n_z.
2026-02-06 13:07:47,061 - choreographer.browsers.chromium - INFO - Temporary directory at: C:\Users\Admin\AppData\Local\Temp\tmp16962n_z
2026-02-06 13:07:47,319 - kaleido.kaleido - INFO - Conforming 1 to file:///C:/Users/Admin/AppData/Local/Temp/tmp1ojxzlvl/index.html
2026-02-06 13:07:47,320 - kaleido.kaleido - INFO - Waiting on all navigates
2026-02-06 13:07:48,213 - kaleido.kaleido - INFO - All navigates

🖼️ saved: ./data/RAGAS\runs\s4_law7_rule7_case3_dw0.5_sw0.5_k11.5_dc2200_rerankT\mean\mean_runs_bar.png


2026-02-06 13:07:48,803 - kaleido.kaleido - INFO - Conforming 1 to file:///C:/Users/Admin/AppData/Local/Temp/tmp0jeik_t3/index.html
2026-02-06 13:07:48,804 - kaleido.kaleido - INFO - Waiting on all navigates
2026-02-06 13:07:49,680 - kaleido.kaleido - INFO - All navigates done, putting them all in queue.
2026-02-06 13:07:49,682 - kaleido.kaleido - INFO - Getting tab from queue (has 1)
2026-02-06 13:07:49,683 - kaleido.kaleido - INFO - Got 9604
2026-02-06 13:07:49,684 - kaleido._kaleido_tab - INFO - Processing Per_question_metrics_mean_of_3_runs_2_decimals.png
2026-02-06 13:07:49,685 - kaleido._kaleido_tab - INFO - Sending big command for Per_question_metrics_mean_of_3_runs_2_decimals.png.
2026-02-06 13:07:49,744 - kaleido._kaleido_tab - INFO - Sent big command for Per_question_metrics_mean_of_3_runs_2_decimals.png.
2026-02-06 13:07:49,746 - kaleido.kaleido - INFO - Reloading tab 9604 before return.
2026-02-06 13:07:49,838 - kaleido.kaleido - INFO - Putting tab 9604 back (queue size: 0)

🖼️ saved: ./data/RAGAS\runs\s4_law7_rule7_case3_dw0.5_sw0.5_k11.5_dc2200_rerankT\mean\mean_runs_table.png
✅ MEAN images: ./data/RAGAS\runs\s4_law7_rule7_case3_dw0.5_sw0.5_k11.5_dc2200_rerankT\mean\mean_runs_bar.png ./data/RAGAS\runs\s4_law7_rule7_case3_dw0.5_sw0.5_k11.5_dc2200_rerankT\mean\mean_runs_table.png
CPU times: total: 12 s
Wall time: 13min 43s
